Here I modify my code to make the slack bus selectable.

Recall that there are four components of an instance of the temporal instanton problem:

* $n$, $n_r$, and $T$

* $Q_x$, which depends on desired correlation

* $A$, which depends on admittance matrix $Y$

* $b$, which depends on $G^0$ and $D$

* $Q_\theta$, which depends on $\tau$ and the line we wish to overload

$A$ depends on which bus is chosen to be slack. Here is a modified method that accepts the index of the slack node as an argument and generates an A matrix:

In [46]:
function tmp_inst_A(n,Ridx,T,Y,slack,k)
    """ Generate the power balance constraint A matrix
    from problem dimensions, admittance matrix,
    and generator participation factors.
    Assumes the admittance matrix is n-by-n.
    
    Returns A, which is (n+1)*T-by-(n+nr)*T
    
    * n is the number of nodes in the network
    * Ridx is a vector indicating wind nodes
    * T is the number of time steps
    * Y is the admittance matrix (n-by-n)
    * slack is the index of the slack bus
    * k is the vector of generator participation factors
    """
    
    # zero out slack row of Y:
    adm = deepcopy(Y)
    adm[slack,:] = zeros(n)
    
    # remove slack column of Y:
    adm = adm[:,setdiff(1:n,slack)]
    
    # A has a block diagonal pattern where each
    # block is Atemp:
    Atemp = zeros(n+1,2*n)
    Atemp = [[-eye(n) adm -k]; ones(1,n) zeros(1,n-1) 1]
    
    # Remove columns corresponding to non-wind nodes
    # and slack node:
    Atemp = sparse(Atemp[:,[Ridx,n+1:2*n]])
    
    # Now we can tile the Atemp matrix to generate A:
    A = Atemp
    for t = 2:T
        A = blkdiag(A, Atemp)
    end
    
    return A
end

tmp_inst_A (generic function with 1 method)

In [50]:
# Test generateA and generateb:
n = 3
Ridx = [1,2,3]
T = 2
Y = [2 3 4;
    3 4 3;
    2 3 4]
slack = 3
k = [0.25,0.25,0.5]

A = tmp_inst_A(n,Ridx,T,Y,slack,k)
full(A)

8x12 Array{Float64,2}:
 -1.0   0.0   0.0  2.0  3.0  -0.25   0.0   0.0   0.0  0.0  0.0   0.0 
  0.0  -1.0   0.0  3.0  4.0  -0.25   0.0   0.0   0.0  0.0  0.0   0.0 
  0.0   0.0  -1.0  0.0  0.0  -0.5    0.0   0.0   0.0  0.0  0.0   0.0 
  1.0   1.0   1.0  0.0  0.0   1.0    0.0   0.0   0.0  0.0  0.0   0.0 
  0.0   0.0   0.0  0.0  0.0   0.0   -1.0   0.0   0.0  2.0  3.0  -0.25
  0.0   0.0   0.0  0.0  0.0   0.0    0.0  -1.0   0.0  3.0  4.0  -0.25
  0.0   0.0   0.0  0.0  0.0   0.0    0.0   0.0  -1.0  0.0  0.0  -0.5 
  0.0   0.0   0.0  0.0  0.0   0.0    1.0   1.0   1.0  0.0  0.0   1.0 